In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import ast

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('./dataset/dataset.csv')
symptoms = pd.read_csv('./dataset/Symptom-severity.csv')
d = data.copy()

from sklearn.preprocessing import LabelEncoder

tmp = data['Disease'].unique()
le = LabelEncoder()
le.fit(tmp)
data['Disease'] = le.transform(data['Disease'])
tmp_encoded = le.transform(tmp)

disease_data = pd.DataFrame({'Disease':tmp, 'Encoding':tmp_encoded})

In [3]:
data = data.applymap(lambda x: x.replace(" ","") if isinstance(x, str) else x) #remove space from in front of each element

df = pd.DataFrame({'Disease':data['Disease']})
all_symptomps = symptoms['Symptom'].unique()

s = data.drop('Disease', axis=1)

for sym in all_symptomps:
    s[sym] = s.apply(lambda r: int(sym in r.values), axis=1)

s = s[all_symptomps]

df = pd.concat([df, s], axis=1)

In [21]:
import google.generativeai as genai

API_KEY = "api key"
genai.configure(api_key=API_KEY)

instructions = f"You are a Doctor. Your task is to retrieve symptoms, match them to this list: {all_symptomps}, and return just python list of the symptoms without anything else."

ai_model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=instructions)

In [31]:
x = df[all_symptomps]
y = df['Disease']
num_classes = len(np.unique(y))
y = tf.keras.utils.to_categorical(y, num_classes)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


"""from sklearn.decomposition import PCA
pca = PCA(n_components=20, random_state=42)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)"""

'from sklearn.decomposition import PCA\npca = PCA(n_components=20, random_state=42)\nx_train = pca.fit_transform(x_train)\nx_test = pca.transform(x_test)'

In [32]:
"""for s in all_symptomps:
    x_train[s] = x_train[s] * (symptoms[symptoms['Symptom'] == s]['weight']).values[0]
    x_test[s] = x_test[s] * (symptoms[symptoms['Symptom'] == s]['weight']).values[0]"""

"for s in all_symptomps:\n    x_train[s] = x_train[s] * (symptoms[symptoms['Symptom'] == s]['weight']).values[0]\n    x_test[s] = x_test[s] * (symptoms[symptoms['Symptom'] == s]['weight']).values[0]"

In [33]:
model = Sequential([
    layers.Dense(128, input_dim=x_train.shape[1], activation='relu'),  # Input layer
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(64, activation='relu'),  # Hidden layer
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    verbose=1)

Epoch 1/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1632 - loss: 3.4597 - val_accuracy: 0.9937 - val_loss: 1.7503
Epoch 2/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7563 - loss: 1.5502 - val_accuracy: 1.0000 - val_loss: 0.1792
Epoch 3/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9254 - loss: 0.4869 - val_accuracy: 1.0000 - val_loss: 0.0370
Epoch 4/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9611 - loss: 0.2578 - val_accuracy: 1.0000 - val_loss: 0.0130
Epoch 5/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9708 - loss: 0.1550 - val_accuracy: 1.0000 - val_loss: 0.0061
Epoch 6/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9779 - loss: 0.1290 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 7/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9861 - loss: 0.0870 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 8/100
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9873 - loss: 0.0762 - val_accuracy: 1.0000 - v

In [34]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 1.0000 - loss: 8.3486e-08
Test Accuracy: 1.00


In [52]:
d[d['Disease'] == 'Diabetes ']

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
70,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,Diabetes,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Diabetes,fatigue,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,Diabetes,fatigue,weight_loss,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4722,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4763,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4804,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4845,Diabetes,fatigue,weight_loss,restlessness,lethargy,irregular_sugar_level,blurred_and_distorted_vision,obesity,excessive_hunger,increased_appetite,polyuria,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
response = ai_model.generate_content("Nowadays, I feel very tired. I also feel hungry very often. I eat more than usual but Im losing weight a lot. My vision is also getting blurry")
sp = ast.literal_eval(response.text)
print(sp)

['fatigue', 'excessive_hunger', 'weight_loss', 'blurred_and_distorted_vision']


In [57]:
t_d = {}
user_symptoms = sp
for s in all_symptomps:
    if s in user_symptoms:
        t_d[s] = 1
    else:
        t_d[s] = 0

test = pd.DataFrame([t_d])
test_pred = model.predict(test)
pred_idx = np.argmax(test_pred)
disease_data[disease_data['Encoding'] == pred_idx]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


,Disease,Encoding
14,Jaundice,28


In [63]:
diagnosis = disease_data[disease_data['Encoding'] == pred_idx]['Disease'].values[0]
output_instructions = """
You are an AI Doctor that uses a neural network model to diagnose a disease. VERY IMPORTANT : Before any diagnosis or recommendations, 
always advise that your advice are not fully correct and seeking medical consult is recommended.
The prompt that will be passed to you will be the diagnosis from the model. You will give the Diagnosis/disease and advise the recommended treatement or steps. Be sure to follow the given structure
of the message
"""
output_model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=output_instructions)
response = output_model.generate_content(f"The Diagnosis is {diagnosis}")
print(response.text)

**Disclaimer:** Please remember that I am an AI and cannot provide medical advice.  The following information is for informational purposes only and should not be considered a substitute for professional medical advice.  **It is crucial to seek immediate medical attention from a qualified healthcare provider for any health concerns, including jaundice.**


**Diagnosis:** Jaundice

**Explanation:** Jaundice is a condition characterized by yellowing of the skin and whites of the eyes. It occurs due to a buildup of bilirubin, a yellow pigment, in the blood.  Several factors can cause this buildup, ranging from relatively benign to serious underlying medical conditions.

**Possible Causes (requiring professional diagnosis):**

* **Liver disease:** Hepatitis (viral, alcoholic, autoimmune), cirrhosis, liver cancer.
* **Gallbladder problems:** Gallstones, bile duct obstruction.
* **Pancreatic problems:** Pancreatitis, pancreatic cancer.
* **Blood disorders:** Hemolytic anemia (where red blood